In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

## Load data

In [5]:
data_set = 'Irish_2010'
year = '2010'
method = 'kmeans'
num_clusters = 1
month = 2

# load series (n clusters)
df = pd.read_csv(path + '/result/' + data_set + '/clustering/point/' + method + '/month_' + str(month) + '_series_' + str(num_clusters) + '.csv', header=None)

# weather series
weather_48 = pd.read_csv(path + '/data/weather/weather_' + data_set + '_cleaned.csv')['T'].values
weather_24 = []
for i in range(int(len(weather_48)/2)):
    weather_24.append((weather_48[i*2] + weather_48[i*2+1])/2)
weather_24 = np.array(weather_24)

weather = []
for i in range(12):
    weather.append(weather_24[sum(months[:i])*24:sum(months[:i+1])*24])
weather = np.array(weather)

In [3]:
# Generate calendar series (week)
week = []
for i in range(months[month-1]):
    week_value = datetime.strptime(year + '%02d%02d' %(month, i+1), '%Y%m%d').weekday()
    for _ in range(24):
        week.append(week_value)

week_01 = np.eye(7)[week]

In [4]:
# Generate calendar series (day)
day = []
for i in range(months[month-1]):
    for _ in range(24):
        day.append(_)

day_01 = np.eye(24)[day]

In [5]:
# Generate calendar series (week * day)
weekday_01 = np.zeros((months[month-1] * 24, 168))
for i in range(len(weekday_01)):
    weekday_01[i][week[i]*24+day[i]] = 1

## Construct the training, validation and test set

In [6]:
cluster = 0   # 0, 1, ..., num_clusters-1

# load series (series of a specific cluster)
series = df.values[cluster].reshape(months[month-1]*24, 1).shape

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,662,663,664,665,666,667,668,669,670,671
0,637.91,473.311,383.219,363.884,361.745,382.678,530.279,930.843,1068.049,930.635,...,1239.639,1167.65,1225.524,1366.894,1636.899,1712.417,1648.714,1480.097,1252.569,895.69


## QRA (Step 1)

In [6]:
from keras.layers import Dense
from keras.models import Sequential

Using TensorFlow backend.


In [22]:
# Build linear model
input_dim = 100
model = Sequential([Dense(1, input_shape=(input_dim,))])

In [23]:
# Train
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, Ytrain, epochs=10)

## QRA (Step 2)

In [8]:
def quantile_loss(y_true, y_pred, q):
    tmp1 = (q / 100 - 1) * (y_true - y_pred)
    tmp2 = q / 100 * (y_true - y_pred)
    return K.mean(K.maximum(tmp1, tmp2))

In [11]:
# Build qr model
input_dim = 8
model = Sequential([Dense(1, input_shape=(input_dim,))])

In [ ]:
# Train
model.compile(loss=lambda y_true, y_pred: quantile_loss(y_ture, y_pred, q), optimizer='adam')